# Подготовка инструментов

Импорт необходимых инструментов

In [1]:
%load_ext autoreload
%autoreload 2

Создадим перцептрон со случаными весами для имитации сложной функции

In [2]:
import numpy as np
import pandas as pd

relu = np.vectorize(lambda x: x if x>0 else 0.)
sigm = lambda x: 1./(1+np.exp(-x))

vec_sizes = [45, 45, 45, 20, 15, 1]

def perceptron(xx, vec_sizes):
    
    xx_ext = xx
    generator = np.random.RandomState(43)
    
    for i, new_size in enumerate(vec_sizes):
        xx_ext = np.hstack([np.ones((xx_ext.shape[0], 1)), xx_ext])
        
        weights = generator.normal(0., 1, size=(xx_ext.shape[1], new_size))
        
        xx_ext = relu(xx_ext.dot(weights))
        
    return np.floor(xx_ext.reshape((-1,)))

# Генерация данных, построение модели

Сгенерируем нормально распределенные фичи и с помощью случайного 
перцептрона сымитируем зависимость целевой переменной. Сгенерированный 
датафрейм состоит из признаков, которые вошли в итоговую модель.

Предположим, что факторы feat_3, feat_8, feat_13, feat_27
являются защищенными характеристиками

In [3]:
# Генератор 
gen = np.random.RandomState(42)


# Генерация признаков
X = pd.DataFrame(data=gen.normal(50., 100., size=(30000, 30)),
                 columns=[f'feat_{i}' for i in range(30)],
                 index=[i for i in range(30000)])


# Имитация зависимости
y = pd.Series(data=1*(perceptron(X.values, [60, 32, 12, 5, 1])>0),
              index=[i for i in range(30000)],
              name='target'
             )

# Защищенные характеристики 
fair_factors = ['feat_3', 'feat_8', 'feat_13', 'feat_27']

# Остальные признаки
no_fair_factors = list(set(X.columns) - set(fair_factors))

Сгенеруем защищенные характеристики и "руками" внесем дискриминацию.
С помощью другого случайного перцептрона сымитируем зависимость 
одного защищенного фактора от признаков, не являющихся защищенными характеристиками

In [4]:
# Генерация
X['feat_3'] = gen.randint(18, 40, size=(30000,))

# Внесение "руками" дискриминации
X['feat_3'][y>0]=gen.randint(30, 41, size=(len(X['feat_3'][y>0]),))

# Генерация
X['feat_8'] = gen.choice([0, 1], size=(30000,), p=[0.4, 0.6])

# Внесение "руками" дискриминации
X['feat_8'][y>0]=gen.choice([0, 1], size=(len(X['feat_8'][y>0]),), p=[0.1, 0.9])

# Генерация
X['feat_13'] = gen.uniform(0, 10, size=(30000,))

# Имитация зависимости 
X['feat_27'] = perceptron(X[no_fair_factors].values, vec_sizes)


/var/folders/th/9030k3h14s3ffm4831c1n0mm0000gn/T/ipykernel_3175/739543630.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['feat_3'][y>0]=gen.randint(30, 41, size=(len(X['feat_3'][y>0]),))
/var/folders/th/9030k3h14s3ffm4831c1n0mm0000gn/T/ipykernel_3175/739543630.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['feat_8'][y>0]=gen.choice([0, 1], size=(len(X['feat_8'][y>0]),), p=[0.1, 0.9])


Разобьем данные и закинем их в sampler, также создадим scorer(он необходим для любой валидации, но имеено в тестах по fairness он не используется, поэтому выберем самый обычный)

In [5]:
from sklearn.model_selection import train_test_split
from sbe_vallib.sampler.supervised_sampler import SupervisedSampler
from sbe_vallib.scorer.table_scorer import BinaryScorer

X_train, X_oos, y_train, y_oos = train_test_split(X, y, 
                                                  test_size=0.1, 
                                                  random_state=42,
                                                  stratify=y)
sampler = SupervisedSampler(train={'X': X_train, 'y_true': y_train}, oos={'X': X_oos, 'y_true': y_oos})
scorer = BinaryScorer()

Построим итоговую модель, она должны быть в sklearn-like формате, то есть реализовать методы predict_proba, fit, predict И АТРИБУТ classes_

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

model = RandomForestClassifier()
model.fit(sampler.train['X'], sampler.train['y_true'])
-1 + 2 * roc_auc_score(sampler.oos['y_true'], model.predict_proba(sampler.oos['X'])[:, 1])

0.8493994304482411

# Пайплайн fairness

Создадим object validation и передадим в него конфиг для fairness анализа

In [7]:
from sbe_vallib.validation import Validation

In [8]:
# Первый способ, это передать, просто путь до конфига, но тогда придется, для каждого теста прописать значения для параметра protected_feats
# Не думаю, что это удобно
val = Validation(model=model,
                 sampler=sampler,
                 scorer=scorer,
                 pipeline='../src/sbe_vallib/table/pipelines/fairness_config.xlsx')

In [9]:
# второй способ это считать config.xlxs из файла и затем дополнить получившийся json тем, что нам надо

val = Validation(model=model,
                 sampler=sampler,
                 scorer=scorer,
                 pipeline='../src/sbe_vallib/table/pipelines/fairness_config.xlsx')

for test_key in val.pipeline['tests_desc']:
    val.pipeline['tests_desc'][test_key]['params'].update({'protected_feats': fair_factors})
val.pipeline['tests_desc']['test_indirect_discr']['params']

{'conf_lvl': 0.95,
 'n_bootstrap': 200,
 'random_seed': 1,
 'protected_feats': ['feat_3', 'feat_8', 'feat_13', 'feat_27']}

In [10]:
#третий наиболее удобный это воспользоваться агрументом валидации tests_params. этот словарь будет передаваться в каждый тест с наивысшим приоритетом.
# перед этим конфиг считается, поэтому абсолютно все параметры передавать не надо.

val = Validation(model=model,
                 sampler=sampler,
                 scorer=scorer,
                 pipeline='../src/sbe_vallib/table/pipelines/fairness_config.xlsx',
                 tests_params={'protected_feats': fair_factors, 'min_freq_pos': 0.00, 'min_freq_value': 0.03, 'cat_features': None},
                 exclude_tests=[],
                 store_path='./fairness_results')
print(f'fair_facotrs: {fair_factors}')

fair_facotrs: ['feat_3', 'feat_8', 'feat_13', 'feat_27']


In [11]:
#Возможны warning по поводу, взятия среднего по пустому списку - это нормально, просто при подсчете TPR FPR, получилось так, что есть только один класс
res = val.validate(save_excel=True)

Test: test_indirect_discr started
Test: test_target_rate_delta started
Test: test_tprd_fprd_delta started
Test: test_oppr_priv_ci started
Test: test_delete_protected started


/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/table/fairness/test_tprd_fprd_delta.py:133: RuntimeWarning: Mean of empty slice.
  groups_metric['fpr'].append(source_preds[target == 0].mean())
/Users/azatsultanov/Programming/vallib/vallib_env/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/azatsultanov/Programming/vallib/repo/vallib/src/sbe_vallib/table/fairness/test_tprd_fprd_delta.py:137: RuntimeWarning: Mean of empty slice.
  swap_preds[target == 0].mean())


In [12]:
#Итак у нас получились следующие тесты их же можно увидеть в excel
res.keys()

dict_keys(['test_indirect_discr', 'test_target_rate_delta', 'test_tprd_fprd_delta', 'test_oppr_priv_ci', 'test_delete_protected'])

In [13]:
res['test_indirect_discr']['result_dataframes'][0]

,Защищенная характеристика,Левая граница CI_0.95 Gini,Правая граница CI_0.95 Gini,Защищенная характеристика не проверяется на дискриминацию,Результат теста
0,feat_3,0.273020,0.334902,no,green
1,feat_8,0.108708,0.223950,no,green
2,feat_13,-0.012842,0.026937,no,green
3,feat_27,0.611863,0.659740,yes,red


In [14]:
res['test_target_rate_delta']['result_dataframes'][0]

,Защищенная характеристика,Отн. изменение частоты таргета у угнетаемой группы,Отн. изменение частоты таргета у привилигированной группы,Интервал значений признака угнетаемой группы,Интервал значений признака привилегированной группы,Результат теста
0,feat_3,0.008850,-0.018349,[39-39],[40-40],green
1,feat_8,1.280992,-0.563694,[0-0],[1-1],yellow
2,feat_13,0.000000,0.000000,[3.036-3.3715],[0.3399-0.6682],green
3,feat_27,0.000000,0.000000,[0.0-5207.0],[0.0-5207.0],green


In [15]:
res['test_tprd_fprd_delta']['result_dataframes'][0]

,Защищенная характеристика,Абс. изменение TPRD после перестановки значений,Абс. изменение FPRD после перестановки значений,Интервал значений признака угнетаемой группы,Интервал значений признака привилегированной группы,Результат теста
0,feat_3,0.023907,0.00000,[39-39],[40-40],green
1,feat_8,0.057067,0.06227,[0-0],[1-1],green
2,feat_13,0.000000,0.00000,[3.036-3.3715],[0.3399-0.6682],green
3,feat_27,0.000000,0.00000,[0.0-5207.0],[0.0-5207.0],green


In [16]:
res['test_oppr_priv_ci']['result_dataframes'][0]

,Защищенная характеристика,95.0%-ый дов. интервал для угнетаемой группы,95.0%-ый дов. интервал для привелегированной группы,99.0%-ый дов. интервал для угнетаемой группы,99.0%-ый дов. интервал для привелегированной группы,Интервал значений признака угнетаемой группы,Интервал значений признака привилегированной группы,Результат теста
0,feat_3,"[0.436, 0.741]","невозможно рассчитать, в группе один класс","[0.388, 0.789]","невозможно рассчитать, в группе один класс",[39-39],[40-40],gray
1,feat_8,"[0.838, 0.925]","[0.768, 0.828]","[0.824, 0.939]","[0.759, 0.837]",[0-0],[1-1],green
2,feat_13,"[0.744, 0.95]","[1.0, 1.0]","[0.711, 0.982]","[1.0, 1.0]",[3.036-3.3715],[0.3399-0.6682],green
3,feat_27,"[0.824, 0.871]","[0.824, 0.871]","[0.817, 0.878]","[0.817, 0.878]",[0.0-5207.0],[0.0-5207.0],green


In [17]:
res['test_delete_protected']['result_dataframes'][0]

,Исходная метрика gini,Метрика gini после удаления,Удаленные признаки,Абс. изменение,Отн. изменение
0,0.849399,0.848257,feat_27 feat_8,-0.001142,-0.001345


# Пайплайн fairness для реальных данных 

У вас должны быть:
1) выборки train и OOS
2) используемые признаки - здесь я взял их из модели разработчика
3) обученная вами модель valid_model с методом predict_proba
4) категориальные признаки - если не уверены, оставляйте в вызове метода FairnessValidation cat_features=[]

In [18]:
fair_factors = [
    'sd_gender_cd',
    'sd_age_yrs_frac_nv',
    'sd_age_yrs_comp_nv',
    'sd_client_valid_nflag',
    'sd_stlmnt_type_cd',
    'sd_russian_citizen_nflag',
    'sd_resident_nflag',
    'sd_sbrf_employee_nflag',
    'dep_social_client_nflag',
    'lne_coborrower_nflag',
    'lne_loan_overdue_nflag',
    'prl_employee_dzo_nflag',
    'prl_social_disab_pension_nflag',
    'seg_client_mp_segment_cd',
    'seg_crd_pos_cat_group',
    'seg_crd_trx_segm',
    'seg_crd_trx_subsegm',
    'seg_age_segment',
    'sd_name_age_segment_cd',
    'sd_age_mnth_comp_nv',
    'sd_cb_staff_nflag',
    'client_region']
feats_for_analyse = list(set(model.used_features) & set(fair_factors))

AttributeError: 'RandomForestClassifier' object has no attribute 'used_features'